# gis Packer Overview

In this notebook we go through some of the flagship features of the gis Packer ETL tool. 

## Load Env

In [1]:
# Basic path management
import os

# Import the gis Packer ETL Tool
import gis_packer

In [2]:
# Path to test raster
single_band_path = os.path.join('..','tests','assets','single_band.tif')
three_band_path = os.path.join('..','tests','assets','three_band.tif')

if not os.path.exists(single_band_path) or not os.path.exists(three_band_path):
    raise Exception('Could not load raster from ./tests/ directory')

In [3]:
# Output dir
out_dir = os.path.join('..','tests','assets','temp')

if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

## Info about raster

The gis_packer.utils.raster.info() function prints basic information about a .tif file

In [4]:
gis_packer.utils.raster.info(single_band_path)

Dataset Name : ../tests/assets/single_band.tif

Number of Bands : 1

Bands indexes : (1,)

Min Bounding Box : BoundingBox(left=296511.0, bottom=4098613.6, right=297700.16, top=4102687.0)

Projected units (width, height) : (1189.1599999999744, 4073.399999999907)

Rows: 3285, Columns: 959

Coordinates System : EPSG:32611

Top left corner coordinates: (296511.0, 4102687.0)
Bottom right corner coordinates: (300583.16, 4101499.08)

Pixel width in meters: 1.5576864535768646
Pixel height in meters: 1.5505735140771637

{'count': 1,
 'crs': CRS.from_epsg(32611),
 'driver': 'GTiff',
 'dtype': 'uint16',
 'height': 3285,
 'interleave': 'band',
 'nodata': None,
 'tiled': False,
 'transform': Affine(1.24, 0.0, 296511.0,
       0.0, -1.24, 4102687.0),
 'width': 959}




In [5]:
gis_packer.utils.raster.info(three_band_path)

Dataset Name : ../tests/assets/three_band.tif

Number of Bands : 3

Bands indexes : (1, 2, 3)

Min Bounding Box : BoundingBox(left=64.29197572679209, bottom=31.541017435305758, right=64.40192951756832, top=31.644054198394265)

Projected units (width, height) : (0.10995379077623113, 0.1030367630885074)

Rows: 1147, Columns: 1224

Coordinates System : EPSG:4326

Top left corner coordinates: (64.29197572679209, 31.644054198394265)
Bottom right corner coordinates: (64.39492265835219, 31.534190239146447)

Pixel width in meters: 11.739319965126416
Pixel height in meters: 10.0

{'count': 3,
 'crs': CRS.from_epsg(4326),
 'driver': 'GTiff',
 'dtype': 'uint8',
 'height': 1147,
 'interleave': 'pixel',
 'nodata': None,
 'tiled': False,
 'transform': Affine(8.983152841195215e-05, 0.0, 64.29197572679209,
       0.0, -8.983152841195215e-05, 31.644054198394265),
 'width': 1224}




## Display Raster

The gis_packer.utils.raster.show() and gis_packer.utils.raster.show_chunk() functions display the raster

In [6]:
?gis_packer.utils.raster.show_chunk

Signature:
gis_packer.utils.raster.show_chunk(
    src_path,
    chunk_size=512,
    offset_x=0,
    offset_y=0,
    block_size=512,
)
Docstring: Preview .tif raster without crashing computer
File:      /usr/local/lib/python3.8/dist-packages/gis_packer-0.0.1-py3.8.egg/gis_packer/utils/raster.py
Type:      function


In [7]:
gis_packer.utils.raster.show(three_band_path)

In [8]:
# only show a 500x500 pixel chunk of the raster
gis_packer.utils.raster.show_chunk(three_band_path, chunk_size=500, offset_x=0)

## Compress Image

It can be useful to compress raster to a lossy format when full resolution is not needed

In [9]:
out_path = os.path.join(out_dir, os.path.basename(three_band_path).replace('.tif', '_compressed.tif'))
gis_packer.utils.raster.compress(three_band_path, out_path)

(Initial size, Final Size, Ratio) : (4.2 MB, 491.3 kB, 11.65%)



In [10]:
gis_packer.utils.raster.show(out_path)

## Remap pixels to uint8

.tif raster often uses 16 bytes (i.e. 0-65535) for pixel values, we can scale it down to 8 bytes (i.e 0-255)

In [11]:
?gis_packer.utils.raster.to_uint8

Signature:
gis_packer.utils.raster.to_uint8(
    src_path,
    out_path,
    min_pixel_value=None,
    max_pixel_value=None,
)
Docstring: Takes an image and converts the data type to uint8 by scaling down the pixel values
File:      /usr/local/lib/python3.8/dist-packages/gis_packer-0.0.1-py3.8.egg/gis_packer/utils/raster.py
Type:      function


In [12]:
out_path = os.path.join(out_dir, os.path.basename(three_band_path).replace('.tif', '_uint8.tif'))
gis_packer.utils.raster.to_uint8(three_band_path, out_path)

Using (5/221) as (min/max) pixel value


In [13]:
gis_packer.utils.raster.show(out_path)

## 

## Reproject

.tif raster can use a wide range of coordinates reference systems (crs), it can be useful to reproject an image to a different crs

In [14]:
?gis_packer.utils.raster.reproject

Signature: gis_packer.utils.raster.reproject(src_path, out_path, target_crs='4326')
Docstring:
Reprojects the raster to a new coordinate system

In order to translate pixel coordinates in a raster dataset into coordinates that use a
spatial reference system, an **affine transformation** must be applied to the dataset.
This **transform** is a matrix used to translate rows and columns of pixels into (x,y)
spatial coordinate pairs. Every spatially referenced raster dataset has an affine transform
that describes its pixel-to-map-coordinate transformation.

In order to reproject a raster dataset from one coordinate reference system to another,
rasterio uses the **transform** of the dataset: this can be calculated automatically using
rasterio's `calculate_default_transform` method:

target CRS: rasterio will accept any CRS that can be defined using WKT
File:      /usr/local/lib/python3.8/dist-packages/gis_packer-0.0.1-py3.8.egg/gis_packer/utils/raster.py
Type:      function


In [15]:
# If we refer to the "Info" section, we can see that our single band raster uses the EPSG:32611 crs
# Lets change it to 4326
out_path = os.path.join(out_dir, os.path.basename(single_band_path).replace('.tif', '_reproj_to_4326.tif'))
gis_packer.utils.raster.reproject(single_band_path, out_path, target_crs='4326')

In [16]:
gis_packer.utils.raster.show(single_band_path)

In [17]:
gis_packer.utils.raster.show(out_path)

## Tiling

Remote sensing can produce extremely huge raster files. For many applications we often have to shard the source raster in small tiles

In [18]:
?gis_packer.utils.raster.create_tiles

Signature:
gis_packer.utils.raster.create_tiles(
    src_path,
    out_dir,
    tile_length_in_m=None,
    tile_length_in_pixels=None,
    tile_overlap=0.0,
)
Docstring:
Function to tile an image into smaller square chunks with embedded georeferencing info
allowing an end user to specify the size of the tile, the overlap of each tile, and when to discard
a tile if it contains blank datasets
Args:
    - src_path (str): path to source image
    - out_dir (str): path to where to save the tiles
    - tile_length_in_m (int): tile height and width in relative ground meters
    - tile_length_in_pixels (int): tile height and width in pixels
    - tile_overlap (float): amount of overlap of each tile in float format.  Should range between 0 and <1.
File:      /usr/local/lib/python3.8/dist-packages/gis_packer-0.0.1-py3.8.egg/gis_packer/utils/raster.py
Type:      function


In [18]:
# Prepare output dir
out_dir_meters = os.path.join(out_dir, 'tiles_in_meters')
out_dir_pixels = os.path.join(out_dir, 'tiles_in_pixels')

if not os.path.isdir(out_dir_meters):
    os.mkdir(out_dir_meters)
    
if not os.path.isdir(out_dir_pixels):
    os.mkdir(out_dir_pixels)

#### Tiling using the ground distance in meters

In [19]:
gis_packer.utils.raster.create_tiles(three_band_path, out_dir_meters, tile_length_in_m=1000)

  8%|▊         | 12/156 [00:00<00:01, 118.29it/s]

Number of tiles = 156, using tile length = 100 pixels / 1000 meters


100%|██████████| 156/156 [00:01<00:00, 129.23it/s]


#### Tiling using a pixel size

In [20]:
gis_packer.utils.raster.create_tiles(three_band_path, out_dir_pixels, tile_length_in_pixels=200)

 29%|██▊       | 12/42 [00:00<00:00, 116.13it/s]

Number of tiles = 42, using tile length = 200 pixels / 2000 meters


100%|██████████| 42/42 [00:00<00:00, 105.52it/s]


In [21]:
gis_packer.utils.raster.show(os.path.join(out_dir_meters, '1.tif'))

In [22]:
gis_packer.utils.raster.show(os.path.join(out_dir_pixels, '1.tif'))

## Database

gis packer can also be connected to a gis database, here we show how we can use the search tool